In [2]:
#pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [35]:
import cv2
import torch
from PIL import Image
import os
import pandas as pd


In [33]:
img_directory = '/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits4.1/test/images/'

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_best_serengeti_splits4.pt')#, source='local')

#Images
imgs = []
img_names = []

i = 1
for img_name in os.listdir(img_directory):
    img = cv2.imread(img_directory+img_name)[:, :, ::-1]
    imgs.append(img)
    img_names.append(img_name)
    
    if i == 20:
        break
    i+=1
    
# Inference
results = model(imgs, size=329)  # includes NMS





Using cache found in /Users/sleung2/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-27 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 224 layers, 7078183 parameters, 0 gradients
Adding AutoShape... 
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [40]:
i = 0

for df,image_name  in zip(results.pandas().xyxy, img_names):
    int_results_df = results.pandas().xyxy[i]
    int_results_df['image_name'] = image_name
    
    if i == 0:
        full_results_df = int_results_df
        
    else:
        full_results_df = pd.concat([full_results_df, 
                                    int_results_df])
        
    i+=1

In [41]:
full_results_df

,xmin,ymin,xmax,ymax,confidence,class,name,image_name
0,31.410490,178.841827,151.594894,328.401520,0.885434,0,bear,SSWI000000019581852B.jpg
0,62.571484,203.640289,120.280609,242.894440,0.768376,8,turkey,SSWI000000017053464A.jpg
1,212.022507,183.643036,234.375885,227.608002,0.315408,8,turkey,SSWI000000017053464A.jpg
0,5.090384,147.796921,67.651543,206.842590,0.800204,3,deer,SSWI000000017956566C.jpg
1,202.590714,165.541428,296.952728,329.000000,0.703804,3,deer,SSWI000000017956566C.jpg
0,81.606148,269.321686,171.232224,328.968781,0.870718,8,turkey,SSWI000000020365431C.jpg
1,280.426422,244.985870,322.570770,272.484039,0.826547,8,turkey,SSWI000000020365431C.jpg
0,0.000000,0.000000,328.475006,329.000000,0.946496,4,elk,SSWI000000020143634C.jpg
0,0.047390,223.555679,54.179619,307.542419,0.884839,7,raccoon,SSWI000000017771284A.jpg
0,178.053467,199.528564,219.470566,267.578247,0.879858,5,foxgray_foxred,SSWI000000022210008C.jpg
